In [2]:
# --- Jupyter Dash setup ---
from jupyter_dash import JupyterDash
JupyterDash.infer_jupyter_proxy_config()

# --- Dash / components ---
import dash_leaflet as dl
from dash import dcc, html
from dash import dash_table
from dash.dependencies import Input, Output
import plotly.express as px

# --- Std libs / data science ---
import base64
import pandas as pd
import numpy as np

# --- Connect CRUD module ---
from CRUD_Python_Module import AnimalShelter

# ---------- MODEL ----------
username = "aacuser"
password = "SNHU1234"

db = AnimalShelter(username, password)

# Helper: read into DataFrame (drop _id for Dash table compatibility)
def read_df(query: dict):
    df = pd.DataFrame.from_records(db.read(query))
    if "_id" in df.columns:
        df = df.drop(columns=["_id"])
    return df

# Unfiltered view (default)
df_all = read_df({})

# ---------- FILTER LOGIC ----------
AGE_FIELD = "age_upon_outcome_in_weeks"
BREED_FIELD = "breed"
SEX_FIELD = "sex_upon_outcome"
LAT_FIELD = "location_lat"
LON_FIELD = "location_long"
NAME_FIELD = "name"

def build_query(rescue: str) -> dict:
    if rescue == "water":
        breeds = [
            "Labrador Retriever Mix",
            "Chesapeake Bay Retriever",
            "Newfoundland",
        ]
        sex = "Intact Female"
        min_w, max_w = 26, 156
    elif rescue == "mountain":
        breeds = [
            "German Shepherd",
            "Alaskan Malamute",
            "Old English Sheepdog",
            "Siberian Husky",
            "Rottweiler",
        ]
        sex = "Intact Male"
        min_w, max_w = 26, 156
    elif rescue == "disaster":
        breeds = [
            "Doberman Pinscher",
            "German Shepherd",
            "Golden Retriever",
            "Bloodhound",
            "Rottweiler",
        ]
        sex = "Intact Male"
        min_w, max_w = 20, 300
    else:
        return {}

    return {
        BREED_FIELD: {"$in": breeds},
        SEX_FIELD: sex,
        AGE_FIELD: {"$gte": min_w, "$lte": max_w},
        LAT_FIELD: {"$ne": None},
        LON_FIELD: {"$ne": None},
    }

# ---------- VIEW ----------
app = JupyterDash(__name__)

# Brand assets
logo_path = "Grazioso Salvare Logo.png"
with open(logo_path, "rb") as f:
    encoded_image = base64.b64encode(f.read()).decode()

brand_header = html.Div(
    [
        html.A(
            html.Img(
                src=f"data:image/png;base64,{encoded_image}",
                style={"height": "90px"},
                alt="Grazioso Salvare",
                title="Visit SNHU",
            ),
            href="https://www.snhu.edu",
            target="_blank",
        ),
        html.Div(
            [
                html.H1("Grazioso Salvare — Search & Rescue Dashboard", style={"margin": "0"}),
                html.P(
                    "Built by Brianna Reed • CS-340 Project Two",
                    style={"margin": "0", "fontStyle": "italic"},
                ),
            ],
            style={"marginLeft": "16px"},
        ),
    ],
    style={"display": "flex", "alignItems": "center", "gap": "12px"},
)

# Filter section
filter_widget = html.Div(
    [
        html.H3("Rescue Filters"),
        dcc.RadioItems(
            id="filter-type",
            options=[
                {"label": "Water Rescue", "value": "water"},
                {"label": "Mountain / Wilderness Rescue", "value": "mountain"},
                {"label": "Disaster Rescue / Individual Tracking", "value": "disaster"},
                {"label": "Reset (All)", "value": "reset"},
            ],
            value="reset",
            labelStyle={"display": "block", "margin": "6px 0"},
            inputStyle={"marginRight": "8px"},
        ),
    ],
    style={
        "width": "280px",
        "padding": "12px",
        "border": "1px solid #ddd",
        "borderRadius": "8px",
        "backgroundColor": "#fafafa",
    },
)

# Data table
table = dash_table.DataTable(
    id="datatable-id",
    columns=[{"name": c, "id": c, "deletable": False, "selectable": True} for c in df_all.columns],
    data=df_all.to_dict("records"),
    page_size=10,
    filter_action="native",
    sort_action="native",
    sort_mode="multi",
    row_selectable="single",
    selected_rows=[],
    style_table={"overflowX": "auto"},
    style_cell={"minWidth": "120px", "whiteSpace": "normal"},
    style_header={"fontWeight": "bold"},
    export_format="csv",
)

# Main layout
app.layout = html.Div(
    [
        brand_header,
        html.Hr(),
        html.Div(
            [
                html.Div(filter_widget, style={"flex": "0 0 300px"}),
                html.Div(
                    [
                        html.H3("Animals (Interactive)"),
                        table,
                        html.Br(),
                        html.Div(
                            className="row",
                            style={"display": "flex", "gap": "12px"},
                            children=[
                                html.Div(id="graph-id", className="col", style={"flex": "1 1 50%"}),
                                html.Div(id="map-id", className="col", style={"flex": "1 1 50%"}),
                            ],
                        ),
                    ],
                    style={"flex": "1 1 auto"},
                ),
            ],
            style={"display": "flex", "gap": "16px"},
        ),
        html.Hr(),
        html.P(
            "© Grazioso Salvare — Dashboard prototype for SNHU CS-340",
            style={"textAlign": "center", "fontStyle": "italic"},
        ),
    ],
    style={"padding": "12px 16px"},
)

# ---------- CONTROLLER ----------

# Update table when a rescue filter is selected
@app.callback(
    Output("datatable-id", "data"),
    Output("datatable-id", "columns"),
    Output("datatable-id", "selected_rows"),
    Input("filter-type", "value"),
)
def update_datatable(filter_type):
    q = build_query(filter_type)
    dff = read_df(q)
    cols = [{"name": c, "id": c, "deletable": False, "selectable": True} for c in dff.columns]
    return dff.to_dict("records"), cols, []

# Breed distribution chart
@app.callback(
    Output("graph-id", "children"),
    Input("datatable-id", "derived_virtual_data"),
)
def update_chart(view_data):
    try:
        if not view_data:
            return html.Div("No data to display.")

        dff = pd.DataFrame(view_data)
        if dff.empty or "breed" not in dff.columns:
            return html.Div("No breed data available for chart.")

        counts = dff["breed"].fillna("Unknown").value_counts().nlargest(10).reset_index()
        counts.columns = ["breed", "count"]

        fig = px.bar(
            counts,
            x="breed",
            y="count",
            title="Breed Distribution (Top 10 in Current View)",
            labels={"breed": "Breed", "count": "Count"},
        )
        fig.update_layout(
            margin=dict(l=20, r=20, t=40, b=20),
            xaxis_tickangle=-30,
            plot_bgcolor="#f8f9fa",
            paper_bgcolor="#ffffff",
        )
        return dcc.Graph(figure=fig)

    except Exception as e:
        return html.Div(f"Error generating chart: {e}")

# Highlight selected columns visually
@app.callback(
    Output("datatable-id", "style_data_conditional"),
    Input("datatable-id", "selected_columns"),
)
def update_styles(selected_columns):
    return [{"if": {"column_id": i}, "background_color": "#D2F3FF"} for i in (selected_columns or [])]

# Update map based on selected row
@app.callback(
    Output("map-id", "children"),
    Input("datatable-id", "derived_virtual_data"),
    Input("datatable-id", "derived_virtual_selected_rows"),
)
def update_map(view_data, selected_rows):
    if not view_data:
        return html.Div("No map data.")
    dff = pd.DataFrame(view_data)
    if dff.empty or LAT_FIELD not in dff or LON_FIELD not in dff:
        return html.Div("No geolocation fields present.")

    row = selected_rows[0] if (selected_rows and len(selected_rows) > 0) else 0
    row = max(0, min(row, len(dff) - 1))

    lat = float(dff.iloc[row][LAT_FIELD])
    lon = float(dff.iloc[row][LON_FIELD])
    breed = str(dff.iloc[row].get(BREED_FIELD, "n/a"))
    name = str(dff.iloc[row].get(NAME_FIELD, "n/a"))

    center = [lat if not np.isnan(lat) else 30.75, lon if not np.isnan(lon) else -97.48]

    return dl.Map(
        style={"width": "100%", "height": "480px"},
        center=center,
        zoom=11,
        children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(
                position=[lat, lon],
                children=[
                    dl.Tooltip(breed),
                    dl.Popup([html.H4("Animal"), html.P(f"Name: {name}"), html.P(f"Breed: {breed}")]),
                ],
            ),
        ],
    )

# Run app in JupyterLab
app.run_server(mode="jupyterlab")
